In [21]:
with open('pagecontent.txt', 'r') as file:
    # Read the content of the file
    content = file.read()

filtered_content = "\n".join(line for line in content.splitlines() if line.strip())

In [22]:
import os
from dotenv import load_dotenv

load_dotenv(".env")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [23]:
import openai

openai.api_key = OPENAI_API_KEY
def get_completion(prompt, model="gpt-4"):
    messages = [{"role":"user","content":prompt}]
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages
    )
    return response.choices[0].message['content']


In [24]:
context = f"""{filtered_content}"""

prompt = f"""
Article: {context}
The article was extracted from webpage, so it may contain some irrelevant information. please find the main content about the title then continue. 
You will generate increasingly concise, entity-dense summaries of the above Article.

Repeat the following 2 steps 3 times.

Step 1. Identify 1-3 informative Entities ("; " delimited) from the Article which are missing from the previously generated summary.
Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the Missing Entities.

A Missing Entity is:
- Relevant: to the main story.
- Specific: descriptive yet concise (5 words or fewer).
- Novel: not in the previous summary.
- Faithful: present in the Article.
- Anywhere: located anywhere in the Article.

Guidelines:
- The first summary should be long (4-5 sentences, 120 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~80 words.
- Make every word count: rewrite the previous summary to improve flow and make space for additional entities.
- Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
- The summaries should become highly dense and concise yet self-contained, e.g., easily understood without the Article.
- Missing entities can appear anywhere in the new summary.
- Never drop entities from the previous summary. If space cannot be made, add fewer new entities.

Remember, use the exact same number of words for each summary.

Answer in JSON. The JSON should be a list (length 3) of dictionaries whose keys are "Missing_Entities" and "Denser_Summary".
"""

response = get_completion(prompt)


In [25]:
import json

json_data = json.loads(response)

with open("output.json", 'w') as json_file:
    json.dump(json_data, json_file, indent=2) 